*Universidad de Buenos Aires >> Facultad de Ciencias Exactas y Naturales >> Ciencias de Datos
Laboratorio de Datos - Primer Cuatrimestre 2024*
# **Trabajo Práctico N° 2**

## **Turno noche - Grupo 2**

**Integrantes:**
*   César Alejandro Momberg
*   Matías Gabriel Resnik

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## **Preprocesamiento de Datos**

1. Cargar en un DataFrame los datos del archivo FBRef2020-21.csv.  

In [2]:
df_torneos_original=pd.read_csv("FBRef2020-21.csv")
df_torneos_original

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PrgDistCarry/90,ProgCarry/90,CarryIntoThird/90,CarryIntoBox/90,Miscontrol/90,Dispossessed/90,PassTarget/90,PassesReceived/90,PassRec%,ProgPassReceived/90
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,29.0,1990.0,22,20,1777,...,96.8,5.79,1.52,0.46,0.66,0.81,42.3,39.3,92.8,2.34
1,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,32.0,1987.0,33,33,2889,...,130.5,1.81,0.25,0.03,0.25,0.47,43.8,42.5,97.0,0.22
2,Pape Abou Cisse,sn SEN,DF,Saint-Étienne,fr Ligue 1,24.0,1995.0,14,14,1260,...,117.9,1.14,0.07,0.00,0.14,0.07,33.7,32.9,97.7,0.00
3,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34.0,1986.0,14,14,1222,...,92.7,3.24,0.66,0.00,0.22,0.00,34.8,33.4,96.0,0.59
4,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32.0,1988.0,32,32,2813,...,166.3,3.71,1.02,0.03,0.45,0.26,50.9,49.5,97.3,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,Reinhold Yabo,de GER,"MF,FW",Arminia,de Bundesliga,28.0,1992.0,13,8,647,...,76.3,3.47,1.39,0.42,0.97,0.97,31.1,21.0,67.4,2.78
2819,Amin Younes,de GER,"MF,FW",Eint Frankfurt,de Bundesliga,26.0,1993.0,26,16,1391,...,138.4,8.71,2.00,1.03,2.19,2.58,58.1,44.3,76.2,8.26
2820,Mattia Zaccagni,it ITA,"MF,FW",Hellas Verona,it Serie A,25.0,1995.0,36,33,2609,...,141.8,7.45,2.52,1.10,2.31,2.86,49.2,35.5,72.1,8.21
2821,Arber Zeneli,xk KVX,"MF,FW",Reims,fr Ligue 1,25.0,1995.0,28,14,1289,...,102.0,5.24,1.40,1.12,2.94,2.45,40.3,28.0,69.3,6.15


2. Eliminar a los jugadores que jugaron menos de 500 minutos en la temporada (columna Min).

In [3]:
jugaron_menos_500_min=df_torneos_original.query("Min<500").index
df_torneos=df_torneos_original.drop(jugaron_menos_500_min)
df_torneos

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PrgDistCarry/90,ProgCarry/90,CarryIntoThird/90,CarryIntoBox/90,Miscontrol/90,Dispossessed/90,PassTarget/90,PassesReceived/90,PassRec%,ProgPassReceived/90
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,29.0,1990.0,22,20,1777,...,96.8,5.79,1.52,0.46,0.66,0.81,42.3,39.3,92.8,2.34
1,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,32.0,1987.0,33,33,2889,...,130.5,1.81,0.25,0.03,0.25,0.47,43.8,42.5,97.0,0.22
2,Pape Abou Cisse,sn SEN,DF,Saint-Étienne,fr Ligue 1,24.0,1995.0,14,14,1260,...,117.9,1.14,0.07,0.00,0.14,0.07,33.7,32.9,97.7,0.00
3,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34.0,1986.0,14,14,1222,...,92.7,3.24,0.66,0.00,0.22,0.00,34.8,33.4,96.0,0.59
4,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32.0,1988.0,32,32,2813,...,166.3,3.71,1.02,0.03,0.45,0.26,50.9,49.5,97.3,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,Reinhold Yabo,de GER,"MF,FW",Arminia,de Bundesliga,28.0,1992.0,13,8,647,...,76.3,3.47,1.39,0.42,0.97,0.97,31.1,21.0,67.4,2.78
2819,Amin Younes,de GER,"MF,FW",Eint Frankfurt,de Bundesliga,26.0,1993.0,26,16,1391,...,138.4,8.71,2.00,1.03,2.19,2.58,58.1,44.3,76.2,8.26
2820,Mattia Zaccagni,it ITA,"MF,FW",Hellas Verona,it Serie A,25.0,1995.0,36,33,2609,...,141.8,7.45,2.52,1.10,2.31,2.86,49.2,35.5,72.1,8.21
2821,Arber Zeneli,xk KVX,"MF,FW",Reims,fr Ligue 1,25.0,1995.0,28,14,1289,...,102.0,5.24,1.40,1.12,2.94,2.45,40.3,28.0,69.3,6.15


3. Eliminar los datos faltantes. Por ejemplo, eliminar columnas con más de 100 datos faltantes y
luego las filas con datos faltantes, o convertir los datos faltantes a algún valor apropiado.

In [4]:
# Se eliminan las columnas que tienen mas de 100 filas con valores faltantes:
columnas_valores_nan= df_torneos.isna().sum()
df_torneos.drop(columns=columnas_valores_nan[columnas_valores_nan>100].index,inplace=True)

# Se cuenta la cantidad de filas que tienen alguna columna con valor faltante:
print("¿Hay filas con valores faltantes?")
print((df_torneos.isna().sum(axis=1)>0).value_counts())

# Al ser poca la cantidad de filas (60), se podrían eliminar o completar con algún valor
# Se decide completar con el valor 0
df_torneos.fillna(0,inplace=True)

# Se verifica que no quedan filas con valores faltantes
print("---------------------------------\n")
print("¿Quedan filas con valores faltantes?")
print((df_torneos.isna().sum(axis=1)>0).value_counts())


¿Hay filas con valores faltantes?
False    1902
True       60
Name: count, dtype: int64
---------------------------------

¿Quedan filas con valores faltantes?
False    1962
Name: count, dtype: int64


4. Al finalizar la limpieza de datos, resetear los índices.

In [5]:
df_torneos.reset_index(drop=True,inplace=True)
df_torneos

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PrgDistCarry/90,ProgCarry/90,CarryIntoThird/90,CarryIntoBox/90,Miscontrol/90,Dispossessed/90,PassTarget/90,PassesReceived/90,PassRec%,ProgPassReceived/90
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,29.0,1990.0,22,20,1777,...,96.8,5.79,1.52,0.46,0.66,0.81,42.3,39.3,92.8,2.34
1,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,32.0,1987.0,33,33,2889,...,130.5,1.81,0.25,0.03,0.25,0.47,43.8,42.5,97.0,0.22
2,Pape Abou Cisse,sn SEN,DF,Saint-Étienne,fr Ligue 1,24.0,1995.0,14,14,1260,...,117.9,1.14,0.07,0.00,0.14,0.07,33.7,32.9,97.7,0.00
3,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34.0,1986.0,14,14,1222,...,92.7,3.24,0.66,0.00,0.22,0.00,34.8,33.4,96.0,0.59
4,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32.0,1988.0,32,32,2813,...,166.3,3.71,1.02,0.03,0.45,0.26,50.9,49.5,97.3,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,Reinhold Yabo,de GER,"MF,FW",Arminia,de Bundesliga,28.0,1992.0,13,8,647,...,76.3,3.47,1.39,0.42,0.97,0.97,31.1,21.0,67.4,2.78
1958,Amin Younes,de GER,"MF,FW",Eint Frankfurt,de Bundesliga,26.0,1993.0,26,16,1391,...,138.4,8.71,2.00,1.03,2.19,2.58,58.1,44.3,76.2,8.26
1959,Mattia Zaccagni,it ITA,"MF,FW",Hellas Verona,it Serie A,25.0,1995.0,36,33,2609,...,141.8,7.45,2.52,1.10,2.31,2.86,49.2,35.5,72.1,8.21
1960,Arber Zeneli,xk KVX,"MF,FW",Reims,fr Ligue 1,25.0,1995.0,28,14,1289,...,102.0,5.24,1.40,1.12,2.94,2.45,40.3,28.0,69.3,6.15


5. Definir el DataFrame **data_num** que solo contenga las variables numéricas, a partir de la
columna *Ast/90*, inclusive. Para clustering y clasificación no vamos a utilizar las variables
categóricas ni edad ni minutos jugados.

In [6]:
# Se busca la posición de la columna 'Ast/90'
array=np.array(df_torneos.columns)
pos=np.where(array=='Ast/90')[0].item()

# Se eliminan todas las columnas hasta la posición de la columna 'Ast/90', el dataFrame resultado se asigna a data_num
data_num= df_torneos.drop(columns=array[0:pos])
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Columns: 111 entries, Ast/90 to ProgPassReceived/90
dtypes: float64(106), object(5)
memory usage: 1.7+ MB


In [7]:
# Se verifica que quedan 5 columnas con valores de tipo object, se eliminan:
data_num.drop(columns=data_num.columns[data_num.dtypes=='object'],inplace=True)
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Columns: 106 entries, Ast/90 to ProgPassReceived/90
dtypes: float64(106)
memory usage: 1.6 MB


## **Clasificación**

7. Ahora queremos poder predecir la posición en la que juega cada jugador según sus datos
estadísticos utilizando KNN. En la columna **Pos** encontramos la posición de los jugadores.
Para la mayoría de los jugadores se indica una única posición pero algunos jugadores tienen
dos posiciones. Para simplificar el análisis vamos a considerar una única posición por jugador.


+   **(a)** Definir la variable *"Pos_filt"* que es la columna *Pos*, pero donde los jugadores deben tener una sola posición (pueden quedarse sólo con la primera posición de cada jugador o eliminar los jugadores con dos posiciones, lo que consideren más conveniente).

In [8]:
# Se obtienen los jugadores que tienen mas de una posición
df_torneos[np.char.str_len(df_torneos["Pos"].values.astype(str))>2]

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PrgDistCarry/90,ProgCarry/90,CarryIntoThird/90,CarryIntoBox/90,Miscontrol/90,Dispossessed/90,PassTarget/90,PassesReceived/90,PassRec%,ProgPassReceived/90
695,Ruben Aguilar,fr FRA,"DF,FW",Monaco,fr Ligue 1,27.0,1993.0,33,27,2225,...,137.4,6.80,2.19,0.20,1.17,1.09,57.6,48.6,84.3,5.06
696,Marc Albrighton,eng ENG,"DF,FW",Leicester City,eng Premier League,30.0,1989.0,31,17,1746,...,95.2,4.54,1.24,0.41,1.13,2.01,47.1,37.1,78.7,3.66
697,Angeliño,es ESP,"DF,FW",RB Leipzig,de Bundesliga,23.0,1997.0,26,24,2161,...,101.0,4.83,1.21,0.58,0.88,0.92,54.7,46.0,84.2,6.25
698,Joe Bryan,eng ENG,"DF,FW",Fulham,eng Premier League,26.0,1993.0,16,7,669,...,117.6,6.22,2.16,0.00,1.49,0.68,42.3,38.4,90.7,1.35
699,Maxwel Cornet,ci CIV,"DF,FW",Lyon,fr Ligue 1,23.0,1996.0,36,29,2446,...,127.6,5.77,1.40,0.44,0.92,0.99,48.4,44.6,92.2,3.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,Reinhold Yabo,de GER,"MF,FW",Arminia,de Bundesliga,28.0,1992.0,13,8,647,...,76.3,3.47,1.39,0.42,0.97,0.97,31.1,21.0,67.4,2.78
1958,Amin Younes,de GER,"MF,FW",Eint Frankfurt,de Bundesliga,26.0,1993.0,26,16,1391,...,138.4,8.71,2.00,1.03,2.19,2.58,58.1,44.3,76.2,8.26
1959,Mattia Zaccagni,it ITA,"MF,FW",Hellas Verona,it Serie A,25.0,1995.0,36,33,2609,...,141.8,7.45,2.52,1.10,2.31,2.86,49.2,35.5,72.1,8.21
1960,Arber Zeneli,xk KVX,"MF,FW",Reims,fr Ligue 1,25.0,1995.0,28,14,1289,...,102.0,5.24,1.40,1.12,2.94,2.45,40.3,28.0,69.3,6.15


In [9]:
# Dada la cantidad de jugadores en esta situación, se decide asignar la primera posición de cada uno
Pos_filt=df_torneos["Pos"].str[0:2]
Pos_filt.name="Pos_filt"
Pos_filt

0       DF
1       DF
2       DF
3       DF
4       DF
        ..
1957    MF
1958    MF
1959    MF
1960    MF
1961    MF
Name: Pos_filt, Length: 1962, dtype: object

*   **(b)** Construir el DataFrame **data_clasif** que resulta de agregarle la columna de *"Pos_filt"* al DataFrame **data_num**.

    Dividir data_num en un 80% para entrenamiento y un 20% para testeo.

In [14]:
# Se genera el DataFrame data_clasif
data_clasif=pd.concat([Pos_filt,data_num], axis=1)

# Se separan las varibles de respuesta (y) de las explicativas (x)
x= data_clasif.drop(["Pos_filt"], axis=1)
y= data_clasif["Pos_filt"]

# Se escalan los datos numéricos para luego poder aplicar K-Nearest Neighbors
X = MinMaxScaler().set_output(transform="pandas").fit_transform(x)

# Se dividen los datos de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

*   **(c)** Aplicar un esquema de validación en el conjunto de entrenamiento para seleccionar el valor óptimo de K. (Esto puede demorar mucho si prueban muchos valores de K, pueden hacerlo hasta un valor máximo de K = 20.)

    **Sugerencia:** puede resultar de ayuda KNeighborsClassifier de sklearn e investigar esa librería para aplicar esquemas de validación.